## NOW WE WILL EXPLORE HOW TO APPLY THESE MODELS WITH EXTRA ADDITIONS TO HAVE A MORE CLEAR OUTCOME



we will have columns in the trained data set that do not exist in the new employees data set so we have to be mindfull of how to add them and alter them

To apply your trained model for predicting attrition and time frame for new employees using the saved models, follow these steps:

After loading the models and loading your new csv file you should :
1. Add a placeholder value for the 'Attrition' column in the new dataset.
2. Preprocess the new data for attrition prediction, ensuring to select the relevant features and scale them using the previously fitted scalers.

X_new_attrition = ('Age', 'DistanceFromHome', 'Education','JobSatisfaction', 'MonthlyIncome', 'PercentSalaryHike','TotalWorkingYears', 'YearsAtCompany', 'YearsSinceLastPromotion',
'PerformanceRating')

X_new_timeframe = ('Age', 'DistanceFromHome', 'Education',
 'JobSatisfaction', 'MonthlyIncome', 'PercentSalaryHike','YearsSinceLastPromotion','PerformanceRating', 'Attrition', 'EnvironmentSatisfaction', 'NumCompaniesWorked','TotalWorkingYears')

3. Predict attrition for the new employees and convert the predictions to binary values.
4. Add the predicted attrition column to the new dataset.
5. Select features for time frame prediction, excluding 'YearsUntilLeave'and Scale the new data for time frame prediction using the previously fitted scaler.
6. Predict the time frame for new employees.
7. Adjust the predicted attrition based on the predicted time frame. (time > 2.5 years attrition is no, else yes)

In [3]:
from keras.models import load_model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

# Load the saved models
model_attrition = joblib.load('/content/best_random_forest_model_ATTRITION.pkl')
model_timeframe = load_model('/content/best_model_timeframe.keras')

# Read the new dataset
new_data = pd.read_csv('/content/machine_test_data.csv')



# Preprocess the new data for prediction
X_new_attrition = new_data[['Age', 'DistanceFromHome', 'Education',
                             'JobSatisfaction', 'MonthlyIncome', 'PercentSalaryHike',
                             'TotalWorkingYears', 'YearsAtCompany', 'YearsSinceLastPromotion',
                             'PerformanceRating']]

# Scale the new data for attrition prediction using the same scaler used for training
scaler_attrition = StandardScaler()
X_new_attrition_scaled = scaler_attrition.fit_transform(X_new_attrition)

# Predict attrition for new employees
attrition_predictions = model_attrition.predict(X_new_attrition_scaled)
# Add a placeholder value for the 'Attrition' column
new_data['Attrition'] = attrition_predictions  # VALUE DOES



# Select features for time frame prediction, excluding 'YearsUntilLeave'
X_new_timeframe = new_data[['Age', 'DistanceFromHome', 'Education',
                     'JobSatisfaction', 'MonthlyIncome', 'PercentSalaryHike', 'YearsSinceLastPromotion',
                     'PerformanceRating', 'Attrition', 'EnvironmentSatisfaction', 'NumCompaniesWorked','TotalWorkingYears']]

# Scale the new data for time frame prediction using the same scaler used for training
scaler_timeframe = StandardScaler()
X_new_timeframe_scaled = scaler_timeframe.fit_transform(X_new_timeframe)

# Predict time frame for new employees
timeframe_predictions = model_timeframe.predict(X_new_timeframe_scaled)

# Threshold for years until leave to determine attrition
threshold_years = 2.5

# Adjust Attrition_Predicted based on Timeframe_Predicted
new_data['Timeframe_Predicted'] = timeframe_predictions
new_data['Attrition_Predicted'] = ['No' if years > threshold_years else 'Yes' for years in new_data['Timeframe_Predicted']]
# Print the new dataset with predicted columns
new_data

4/4 [==============================] - 0s 3ms/step


,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,...,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition,Timeframe_Predicted,Attrition_Predicted
0,28,Travel_Rarely,866,Sales,5,3,Medical,1,1469,4,...,6,4,3,5,4,1,3,0,3.836880,No
1,53,Travel_Rarely,1084,Research & Development,13,2,Medical,1,250,4,...,5,3,3,4,2,1,3,0,4.271675,No
2,24,Travel_Rarely,240,Human Resources,22,1,Human Resources,1,1714,4,...,1,2,3,1,0,0,0,1,2.103166,Yes
3,45,Travel_Rarely,1339,Research & Development,7,3,Life Sciences,1,86,2,...,25,2,3,1,0,0,0,0,15.419602,No
4,36,Travel_Rarely,1396,Research & Development,5,2,Life Sciences,1,304,4,...,16,3,4,13,11,3,7,0,12.374458,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,31,Travel_Rarely,1232,Research & Development,7,4,Medical,1,462,3,...,11,2,2,11,9,4,10,0,10.329585,No
96,29,Travel_Rarely,1246,Sales,19,3,Life Sciences,1,1497,3,...,10,3,3,10,7,0,4,0,9.213132,No
97,37,Non-Travel,1063,Research & Development,25,5,Medical,1,529,2,...,15,2,3,13,11,10,7,0,10.804009,No
98,47,Travel_Rarely,571,Sales,14,3,Medical,1,1503,3,...,11,4,2,5,4,1,2,0,4.360548,No


In [2]:
# Save the new_data DataFrame to a CSV file
new_data.to_csv('/content/new_data_predictions.csv', index=False)

In [4]:
data_subset = pd.read_csv('/content/IBM-HR-Employee-Attrition.csv')

attrition_counts_main_df = data_subset['Attrition'].value_counts()

# Print the counts
print("Attrition Predicted Counts:")
print(attrition_counts_main_df)

Attrition Predicted Counts:
No     1233
Yes     237
Name: Attrition, dtype: int64


In [8]:
new_data = pd.read_csv('/content/new_data_predictions.csv')

attrition_counts_sample_df = new_data['Attrition_Predicted'].value_counts()

# Print the counts
print("Attrition Predicted Counts:")
print(attrition_counts_sample_df)

Attrition Predicted Counts:
No     80
Yes    20
Name: Attrition_Predicted, dtype: int64
